# Initializing the environment and clusters

In [ ]:
import ipyparallel as ipp
c = ipp.Client()
dview = c[:]

In [ ]:
%%px --local
import sys
sys.path.append("/home/sac086/wisdm_model_personalization/")
import warnings
import os
from wisdm import wisdm
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit, ShuffleSplit
from sklearn.cluster import KMeans
from scipy.stats import mode
from collections import Counter
import time

# Experiment #1
In this experiment we compare differing amounts of personal data selected at random to combine with the impersonal 
training set as well as a cluster of the impersonal training set

In [ ]:
%%px --local
experiment_name = "experiment_08-31_train_v1_test_v2_random/"

description = '''
Training and calibrating probability estimations on the WISDM v1.1 dataset. 
Probability estimation was done with stream sampling where those samples whose most probable class label
was did not exceed some threshold (where the threshold is some amount greater than the uniform probability for the class) were
actively labeled.  Testing on WISDM 2.0 with one participant held out'''

experiment_output_path = "/home/sac086/wisdm_model_personalization/results/" + experiment_name

In [ ]:
if not os.path.exists(experiment_output_path):
    os.makedirs(experiment_output_path)
with open(experiment_output_path + "README.md", "w") as fOut:
    fOut.write(description)

In [ ]:
command = """wisdm.pipeline1('%s', user_ids)""" % experiment_output_path

In [ ]:
wisdm.set_data(version="2", make_compatible=True)

In [ ]:
results = dview.execute('print(wisdm.user_ids)')


In [ ]:
results.display_outputs()

In [ ]:
# divide the user ids up among the different cores
scatter_result = dview.scatter("user_ids", wisdm.user_ids)

start = time.time()
dview.block = True
results = dview.execute(command)
finish = time.time()
print("Finished all models in %s minutes" % ((finish - start) / 60.))